In [1]:
# Download the shape predictor if not already present
!wget -nc http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 -k shape_predictor_68_face_landmarks.dat.bz2  # Keep the compressed file

File ‘shape_predictor_68_face_landmarks.dat.bz2’ already there; not retrieving.

bunzip2: Output file shape_predictor_68_face_landmarks.dat already exists.


In [ ]:
# Import required libraries
import cv2
import dlib
import numpy as np
from scipy.spatial import distance
from google.colab import files
from google.colab.patches import cv2_imshow

# Function to calculate the eye aspect ratio (EAR)
def calculate_eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Upload shape_predictor_68_face_landmarks.dat if needed
try:
    predictor_path = "shape_predictor_68_face_landmarks.dat"
    with open(predictor_path, 'r') as f:
        print(f"{predictor_path} is already available.")
except FileNotFoundError:
    print("Upload the shape_predictor_68_face_landmarks.dat file:")
    uploaded = files.upload()
    predictor_path = list(uploaded.keys())[0]

# Load the face detector and facial landmarks predictor from dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Define the eye aspect ratio threshold for drowsiness detection
EAR_THRESHOLD = 0.3

# Open the video file
video_path = "/content/Video1.mp4"
cap = cv2.VideoCapture(video_path)

# Check if the video file was opened successfully
if not cap.isOpened():
    print(f"Error: Could not open video file {video_path}")
else:
    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            print("End of video or unable to read frame.")
            break

        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale frame
        faces = detector(gray)

        for face in faces:
            # Predict the facial landmarks
            shape = predictor(gray, face)
            shape = np.array([[shape.part(i).x, shape.part(i).y] for i in range(68)])

            # Extract the left and right eye coordinates
            left_eye = shape[42:48]
            right_eye = shape[36:42]

            # Calculate the eye aspect ratios
            left_ear = calculate_eye_aspect_ratio(left_eye)
            right_ear = calculate_eye_aspect_ratio(right_eye)

            # Compute the average eye aspect ratio
            avg_ear = (left_ear + right_ear) / 2.0

            # Check if the average eye aspect ratio is below the threshold
            if avg_ear < EAR_THRESHOLD:
                cv2.putText(frame, "Drowsy", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # Draw the eye contours for visualization
            for point in left_eye:
                cv2.circle(frame, tuple(point), 2, (255, 0, 0), -1)
            for point in right_eye:
                cv2.circle(frame, tuple(point), 2, (255, 0, 0), -1)

        # Display the frame
        cv2_imshow(frame)

        # Break loop if user presses 'q' (not applicable in Colab)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video file and close all windows
    cap.release()
    cv2.destroyAllWindows()